#### Import Libraries

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D

#### Define the CNN Model

In [7]:
def create_model(input_shape):
    model = Sequential()

    # Convolutional Layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolutional Layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(GlobalAveragePooling2D())

    # Fully Connected Layer
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Prevent overfitting

    # Output Layer
    model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

    return model


#### Compile the Model

In [8]:
input_shape = (256, 256, 1)  # Grayscale images, hence 1 channel
model = create_model(input_shape)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 global_average_pooling2d_1  (None, 64)                0         
  (GlobalAveragePooling2D)                                       
                                                      

#### Train the Model


In [9]:
import numpy as np

# Load the preprocessed data
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
X_val = np.load('X_val.npy')
y_val = np.load('y_val.npy')


In [10]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,  #adjustable
    batch_size=32,  # Typical batch size
    verbose=1
)

Epoch 1/15
11/11 [==============================] - 21s 2s/step - loss: 0.6457 - accuracy: 0.7425 - val_loss: 0.5662 - val_accuracy: 0.8452
Epoch 2/15
11/11 [==============================] - 16s 2s/step - loss: 0.4616 - accuracy: 0.8832 - val_loss: 0.4494 - val_accuracy: 0.8452
Epoch 3/15
11/11 [==============================] - 16s 1s/step - loss: 0.3653 - accuracy: 0.8832 - val_loss: 0.5016 - val_accuracy: 0.8452
Epoch 4/15
11/11 [==============================] - 15s 1s/step - loss: 0.3854 - accuracy: 0.8832 - val_loss: 0.4448 - val_accuracy: 0.8452
Epoch 5/15
11/11 [==============================] - 15s 1s/step - loss: 0.3807 - accuracy: 0.8832 - val_loss: 0.4461 - val_accuracy: 0.8452
Epoch 6/15
11/11 [==============================] - 15s 1s/step - loss: 0.3892 - accuracy: 0.8832 - val_loss: 0.4620 - val_accuracy: 0.8452
Epoch 7/15
11/11 [==============================] - 15s 1s/step - loss: 0.3695 - accuracy: 0.8832 - val_loss: 0.4483 - val_accuracy: 0.8452
Epoch 8/15
11/11 [==

#### Evaluate the Model

In [11]:
# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Validation Accuracy: 84.52%


#### Save the Model

In [12]:
# Save the trained model
model.save('aneurysm_classifier.h5')

d:\Aneurysm-Detection\aneurysm\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#### Use the Model for Predictions

In [15]:
# Load the saved model
from tensorflow.keras.models import load_model
import cv2
model = load_model('aneurysm_classifier.h5')

# Predict on a new image
def predict_image(model, image):
    image_resized = cv2.resize(image, (256, 256)) / 255.0  # Resize and normalize
    image_expanded = np.expand_dims(image_resized, axis=(0, -1))  # Expand dims for batch & channel
    prediction = model.predict(image_expanded)
    return "Aneurysm Detected" if prediction[0][0] > 0.5 else "No Aneurysm"

# Example usage
new_image = cv2.imread('test.jpg', cv2.IMREAD_GRAYSCALE)
print(predict_image(model, new_image))

1/1 [==============================] - 0s 279ms/step
No Aneurysm
